# Scoring notebook

This is the scoring notebook for the data driven competition at CMF 2022. You can change cells with `### YOUR CODE HERE` line, all other cells are read-only. However, you can add new cells to organize your code in a convenient way.

In [ ]:
!pip install statsmodels
!pip install warnings
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', UserWarning)

Let us load the dataset. Columns in the test (public as well as private) dataset are equivalent to the train dataset.

In [ ]:
dataset = pd.read_csv('dataset.zip', index_col=0, header=[0, 1])
dataset.rename(
    columns={
        'Unnamed: 209_level_1': 'count',
        'Unnamed: 210_level_1': 'price',
    },
    level = 1,
    inplace = True
)
dataset.head()

In [ ]:
class Dataloader():
    def __init__(
        self, 
        dataframe: pd.DataFrame, 
        window_size: int, 
        step_size: int,
        horizon: int,
        first_pred: int
    ):
        self.df = dataframe
        self.window_size = window_size
        self.step_size = step_size
        self.horizon = horizon
        self.first_pred = first_pred
        assert self.first_pred > self.window_size
        feat_idx = []
        target_idx = []
        for i in range(self.first_pred, self.df.shape[0], self.step_size):
            feat_idx.append(range(i-self.horizon-self.window_size+1, i-self.horizon+1))
            target_idx.append(i)
        self.feat_idx = feat_idx
        self.target_idx = target_idx
    
    def __len__(self):
        return len(self.feat_idx)
    
    def __iter__(self):
        self.iter = 0
        return self

    def __next__(self):
        if self.iter < len(self.feat_idx):
            feat = self.df.iloc[self.feat_idx[self.iter]]
            target = self.df.iloc[self.target_idx[self.iter], -1]
            self.iter += 1
            return feat, target
        else:
            raise StopIteration

Column **price** represents the price at moment **t**. The task is to predict **price** values at moment **t+60**.

The forecasting problem is defined as follows. Consider the multivariate time series of features (exogenous variables) $X_0, X_1, \dots $ where $X_i \in \mathbb{R}^d$. Consider the univariate time series of targets (endogenous variables) $y_0, y_1, ...$ where $y_i \in \mathbb{R}$. The task is to predict the $y_{T+h}$ where $T \in \{1000, 1001, \dots\}$ is the last available time stamp and $h = 60$ is the forecasting horizon by the given _sliding window_ over pairs $(X, y)_{T-N+1}, (X, y)_{T-N+2}, \dots, (X, y)_T$ with the selected window size $1 \leq N \leq 1000$. The optimization problem is minimizing the mean squared error between predictions and targets.

Select the window size appropriately to your solution.

In [ ]:
window_size = 1000

In [ ]:
assert (1 <= window_size) and (window_size <= 1000)

The dataloader defines the forecasting problem with the selected window size.

**Remark**: first 1060 observations in both test datasets will not be scored.

In [ ]:
loader = Dataloader(
    dataframe=dataset, 
    window_size=window_size, 
    step_size=1, 
    horizon=60, 
    first_pred=1060)

for feat, target in loader:
    break
feat.shape, target

Define your forecasting model. You can install necessary libraries by `!pip install ... `. You can find installed packages in [requirements](https://github.com/vpozdnyakov/EvalAI/blob/master/requirements/worker.txt). Here is also CPU version of `torch==1.10.2`. Do not train the model here, instead download the weights of a pretrained model from your own cloud service, e.g. google drive by `gdown` as follows:

```python
!pip install gdown==4.2.0 -q
url = ...
gdown.download(url, 'model_scripted.pt', fuzzy=True)
model = torch.jit.load('model_scripted.pt')
```

You can change the template by adding additional methods, parameters, etc.

In [ ]:
class ForecastingModel():
    def __init__(self):
        self.models = None
        self.j = 0
        self.t = []
        
    def init2(self, prices):
        last_ind = prices.index[-1]
        self.models = []
        for j in range(0,60):
            self.t.append(len(prices[j::60])+1)
            model=ARIMA(prices[j::60].values,order=(2,1,1), trend = 't')
            self.models.append(model.fit())
            if prices[j::60].index[-1] == last_ind:
                self.j = j%60
            
    def forecast(self, feat):
        prices = feat['price']

        if not self.models:
            self.init2(prices)
            
            
        self.j = (self.j+1)%60   
        
        if np.isnan(prices.values[-1]):
            result=self.models[self.j].predict(self.t[self.j], self.t[self.j])
            self.models[self.j] = self.models[self.j].append(endog = result, refit = False)
            self.t[self.j]+=1
            return result
        
        self.models[self.j] = self.models[self.j].append(endog = prices.values[-1], refit = False)
            
        result=self.models[self.j].predict(self.t[self.j], self.t[self.j])
        self.t[self.j]+=1   
        
        return result

model = ForecastingModel()

In **forecast** function you can do preprocessing, e.g. deletion unnecessary data or aggregation.

In [ ]:
def forecast(feat):
    
    return model.forecast(feat)

Scoring the model.

In [ ]:
pred = []
target = []
for feat, _target in loader:
    pred.append(forecast(feat))
    target.append(_target)
mse(pred, target)

Let us draw the forecast visualization.

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(target, label='target')
plt.plot(pred, label='forecast')
plt.title('Price of the asset')
plt.legend()
plt.show()

Example of 1000 forecasts.

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(target[-100000:-99000], label='target')
plt.plot(pred[-100000:-99000], label='forecast')
plt.title('Price of the asset')
plt.legend()
plt.show()